In [ ]:
import os

from pathlib import Path
from dotenv import load_dotenv
from typing import Any, Tuple, Literal
from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthClientCredentials
from dataclasses import dataclass

from cognite.client.data_classes.data_modeling import (
    Node,
    NodeId,
    NodeList,
    NodeApply,
    NodeApplyList,
    NodeApplyResultList,
    instances,
    InstancesApplyResult,
    ViewId,
)
from cognite.client.data_classes.filters import (
    Filter,
    Equals,
    In,
    Range,
    Exists,
)

In [ ]:
def create_client(cdf_project, cdf_cluster, client_id, client_secret, tenant_id, debug: bool = False):
    SCOPES = [f"https://{cdf_cluster}.cognitedata.com/.default"]
    TOKEN_URL = (
        f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    )
    creds = OAuthClientCredentials(
        token_url=TOKEN_URL,
        client_id=client_id,
        client_secret=client_secret,
        scopes=SCOPES,
    )
    cnf = ClientConfig(
        client_name="DEV_Working",
        project=cdf_project,
        base_url=f"https://{cdf_cluster}.cognitedata.com",  # NOTE: base_url might need to be adjusted if on PSAAS or Private Link
        credentials=creds,
        debug=debug,
    )
    client = CogniteClient(cnf)
    return client

In [ ]:
# Configuration Classes
@dataclass
class ViewPropertyConfig:
    schema_space: str
    external_id: str
    version: str
    instance_space: str | None = None

    def as_view_id(self) -> ViewId:
        return ViewId(
            space=self.schema_space, external_id=self.external_id, version=self.version
        )

    def as_property_ref(self, property) -> list[str]:
        return [self.schema_space, f"{self.external_id}/{self.version}", property]

In [ ]:
# Connect to cognite project
load_dotenv()
cdf_project=os.getenv("CDF_PROJECT")
cdf_cluster=os.getenv("CDF_CLUSTER")
client_id=os.getenv("IDP_CLIENT_ID")
client_secret=os.getenv("IDP_CLIENT_SECRET")
tenant_id=os.getenv("IDP_TENANT_ID")

cdf_client = create_client(cdf_project, cdf_cluster, client_id, client_secret, tenant_id)

In [ ]:
# Create a mimic of a View
txFile_view: ViewPropertyConfig = ViewPropertyConfig(
    schema_space="sp_enterprise_process_industry",
    external_id="txFile",
    version="v1",
    instance_space="springfield_instances",
)

In [ ]:
# retrieve instances of txFile
txFile_instances: NodeList[Node] = cdf_client.data_modeling.instances.list(instance_type="node", sources=txFile_view.as_view_id(), limit=-1)
print(txFile_instances[0])

In [ ]:
# now we need to enrich the data... -> lets add detectInDiagram and toAnnotate to the tags property
file_node_apply_list: NodeApplyList = txFile_instances.as_write()

for file in file_node_apply_list:
    file.sources[0].properties["tags"] = ["ToAnnotate", "DetectInDiagrams"]
print(file_node_apply_list[0])

In [ ]:
# Great now that we've updated the nodes' tags property, let's apply those changes
cdf_client.data_modeling.instances.apply(
    nodes=file_node_apply_list
)

In [72]:
# Now lets do the same with the equipment nodes in the project so that we have entities to match against
txEquipment_view: ViewPropertyConfig = ViewPropertyConfig(
    schema_space="sp_enterprise_process_industry",
    external_id="txEquipment",
    version="v1",
    instance_space="springfield_instances",
)

In [73]:
# retrieve instances of txEquipment
txEquipment_instances: NodeList[Node] = cdf_client.data_modeling.instances.list(instance_type="node", sources=txEquipment_view.as_view_id(), limit=-1)
print(txEquipment_instances[0])

{
    "space": "springfield_instances",
    "external_id": "WMT:23-PAHH-96156-01",
    "version": 2,
    "last_updated_time": "2025-06-28 04:01:42.648+00:00",
    "created_time": "2025-06-27 23:48:32.796+00:00",
    "instance_type": "node",
    "properties": {
        "sp_enterprise_process_industry": {
            "txEquipment/v1": {
                "name": "23-PAHH-96156-01",
                "description": "VRD - PH 1STSTG COMP OUTER SEAL DE : PRESSURE ALARM HIGH HIGH",
                "files": [],
                "manufacturer": "1686.0",
                "serialNumber": "1.000000000112122E15",
                "sourceId": "769587.0",
                "sourceUpdatedUser": "1001.0"
            }
        }
    }
}


In [74]:
equipment_node_apply_list = txEquipment_instances.as_write()

for equipment in equipment_node_apply_list:
    equipment.sources[0].properties["tags"] = ["DetectInDiagrams"]
print(equipment_node_apply_list[0])

{
    "space": "springfield_instances",
    "external_id": "WMT:23-PAHH-96156-01",
    "instance_type": "node",
    "existing_version": 2,
    "sources": [
        {
            "properties": {
                "name": "23-PAHH-96156-01",
                "description": "VRD - PH 1STSTG COMP OUTER SEAL DE : PRESSURE ALARM HIGH HIGH",
                "files": [],
                "manufacturer": "1686.0",
                "serialNumber": "1.000000000112122E15",
                "sourceId": "769587.0",
                "sourceUpdatedUser": "1001.0",
                "tags": [
                    "DetectInDiagrams"
                ]
            },
            "source": {
                "space": "sp_enterprise_process_industry",
                "external_id": "txEquipment",
                "version": "v1",
                "type": "view"
            }
        }
    ]
}


In [75]:
cdf_client.data_modeling.instances.apply(equipment_node_apply_list)

InstancesApplyResult(nodes=[<NodeApplyResult(space='springfield_instances', external_id='WMT:23-PAHH-96156-01', version=3) at 0x114fb77d0>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-LG-92523-02', version=3) at 0x110f23620>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-EZSM-92501B', version=3) at 0x114ff59a0>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-LY-92521B', version=3) at 0x1150329f0>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-PI-92536', version=3) at 0x1150311c0>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-PDT-92534', version=4) at 0x110e650a0>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-HV-92541-01', version=3) at 0x110e671d0>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-PDAH-92602', version=3) at 0x1149dbd70>, <NodeApplyResult(space='springfield_instances', external_id='WMT:23-TE-96181-01', version=4) at 0x